In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [2]:
url = "https://www.amazon.in/gp/bestsellers/books/"

In [37]:
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)
    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        # print(name)
        n = name.find_all('img', alt=True)
        # print(n)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None and len(n) != 0:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    #print(alls)
    return alls

In [38]:
results = []
for i in range(1, no_pages + 1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [39]:
df = pd.read_csv("amazon_products.csv")

In [40]:
df.shape

(100, 5)

In [41]:
df.head(61)

,Book Name,Author,Rating,Customers_Rated,Price
0,Think Like a Monk,Jay Shetty,3.9 out of 5 stars,10,₹ 391.00
1,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6 out of 5 stars,"3,512",₹ 380.00
2,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"5,784",₹ 499.00
3,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6 out of 5 stars,"2,951",₹ 599.00
4,The Alchemist,Paulo Coelho,4.6 out of 5 stars,"27,413",₹ 199.00
5,World’s Greatest Books For Personal Growth & W...,Various,4.5 out of 5 stars,722,₹ 279.00
6,The Power of your Subconscious Mind,Joseph Murphy,4.5 out of 5 stars,"18,051",₹ 157.00
7,A Brief History of Modern India (2019-2020 Edi...,Spectrum Books Pvt.Ltd.,4.5 out of 5 stars,"2,002",₹ 237.00
8,Oxford Student Atlas for India - Third Edition,Oxford University Press,4.6 out of 5 stars,"1,229",₹ 231.00
9,One Arranged Murder,Chetan Bhagat,-1,0,₹ 175.00
